### Используем НОРМАЛЬНЫЙ апи клиент, который по хорошему надо бы интегрировать в дансвер

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [2]:
from notion_client import Client, AsyncClient

# Initialize a client with the integration token
notion = Client(auth=NOTION_API_KEY)


In [3]:
page_id = NOTION_ROOT_ID

# Retrieve the page
page = notion.pages.retrieve(page_id=page_id)

# Print page properties
print(page["properties"])


{'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Франция | Гайд по иммиграции и интеграции', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Франция | Гайд по иммиграции и интеграции', 'href': None}]}}


In [4]:

pages = notion.databases.query(database_id=NOTION_DATABASE_ID) # корретно влзвращает и страницы и теги, но нужно не проморгать пагинацию (has_more и next_cursor)

In [5]:
page = pages['results'][1]

In [6]:
tag_groups = {}
for page in pages['results']:
    for property in page['properties']:
        if page['properties'][property]['type'] in ["select", "multi_select"]:
            if not property in tag_groups:
                tag_groups[property] = set()
            info = page['properties'][property]
            if info['type'] == 'select':
                tag_groups[property].add(info['select']['name'])
            else:
                tag_groups[property].update([x['name'] for x in info['multi_select']])



In [7]:
remove_list_for_tags = [
    "115",
    "18-25",
    "AMELI",
    "API",
    "APL",
    "APS",
    "CADA",
    "CAF",
    "CIR",
    "CPAM",
    "CROUS",
    "CSS",
    "CVEC",
    "CritAir",
    "EHPAD",
    "FSL",
    "HUDA",
    "MDPH",
    "OFII",
    "OFPRA",
    "OMC",
    "PACS",
    "PASS",
    "PUMa",
    "RSA",
    "SPADA",
    "VAE",
    "VAPP",
    "VES",
    "ZAPI",
    "asso",
]

In [8]:
tag_groups['Tags'] = tag_groups['Tags'] - set(remove_list_for_tags)

In [9]:
tag_groups

{'Tags': {'LGBTQIA+',
  'PhD',
  'entrepreneur',
  'mutuelle',
  'service civique',
  'urgence',
  'visa C',
  'visa D',
  'ВНЖ',
  'ДМС',
  'Дублин',
  'НКО',
  'Скорая',
  'Украина',
  'аборт',
  'адвокат',
  'административная прописка',
  'апостиль',
  'аренда',
  'аспирантура',
  'ассо',
  'банк',
  'банк Франции',
  'барахолки',
  'беженство',
  'беженцы',
  'библиотека',
  'бизнес-план',
  'брак',
  'братство',
  'вакцинация',
  'валидация навыков',
  'водительские права',
  'воссоединение семьи',
  'временная защита',
  'временное жилье',
  'выборы',
  'гарант',
  'гражданская служба',
  'граница',
  'декларация',
  'дети',
  'детский сад',
  'доверенность',
  'документы',
  'долгосрочные визы',
  'дом престарелых',
  'досье арендатора',
  'еда',
  'женералист',
  'женщины',
  'животные',
  'жилье',
  'завершение контракта',
  'запрос убежища',
  'заселение',
  'зима',
  'изучение французского языка',
  'иммиграция',
  'иммиграция во Францию',
  'имущество',
  'инвалидность',
  

In [11]:
# not transform to this format {"dtags" : [{"tag_key": "Tags", "all_possible_tag_values" : ["равенство","женералист"]}]}
tag_groups_formatted = [{"tag_key": key, "all_possible_tag_values": list(value)} for key, value in tag_groups.items()]

In [12]:
# dump to single string json compact as possible
import json
tag_groups_formatted_json = json.dumps({"dtags":tag_groups_formatted}, separators=(',', ':'), ensure_ascii=False)
with open('tag_groups.json', 'w') as f:
    f.write(tag_groups_formatted_json)

In [ ]:
page['properties']['Select']['select']['name']

In [ ]:
page["url"]

In [ ]:
page["properties"]["URL"]["url"]

In [ ]:
id_to_tags = {}
for page in pages['results']:
    ms_dicts = page['properties']['Tags']['multi_select']
    tag_names = [tag["name"] for tag in ms_dicts]
    id_to_tags[page['id']] = tag_names

In [ ]:
id_to_tags["f2b1db66-7b1d-4e1d-8c4c-3c8ceb481511"]

In [ ]:
notion.pages.retrieve(page_id=database_id)

In [ ]:
has_more : bool = pages["has_more"]
cursor : str = pages["next_cursor"]

In [ ]:
pages['next_cursor']

In [ ]:
database_id = NOTION_DATABASE_ID
notion.databases.retrieve(database_id=database_id)

In [ ]:
def retrieve_blocks(page_id):
    all_blocks = []
    start_cursor = None
    while True:
        response = notion.blocks.children.list(block_id=page_id, start_cursor=start_cursor)
        blocks = response.get("results", [])
        all_blocks.extend(blocks)
        if not response.get("has_more", False):
            break
        start_cursor = response.get("next_cursor", None)
    return all_blocks

root_page_id = "a283ff3a46804b39b50e29c566369c2f"

# Retrieve blocks from the root page
blocks = retrieve_blocks(root_page_id)
visited_ids = set()

def retrieve_all_blocks(blocks, level=0):
    all_blocks = []
    for block in blocks:
        if block["id"] not in visited_ids:
            #print("  " * level + block["type"], block.get("id", ""))
            all_blocks.append(block)
            visited_ids.add(block["id"])
            if block.get("has_children", False):
                child_blocks = retrieve_blocks(block["id"])
                new_blocks = retrieve_all_blocks(child_blocks, level + 1)
                all_blocks.extend(new_blocks) if new_blocks else None
    return all_blocks

# Start retrieving all nested blocks
all_blocks = retrieve_all_blocks(blocks)


In [ ]:
for i, block in enumerate(all_blocks):
    if "https://www.defenseurdesdroits.fr/carte-des-delegues" in str(block):
        print(block)
        print(i)


In [ ]:
print(all_blocks[131])

### Используем то что уже зашито в дасвер, чтобы ничего не сломать

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [ ]:
from danswer.connectors.notion.connector import NotionConnector

In [ ]:
t1 = {"a" : 5, "b" : 3}
t2 = {"a" : None, "b" : None}
#{**t2,**t1}
t2.update(t1)
t2

In [ ]:
#connector = NotionConnector(root_page_id="f2c93c208bd345038ef7e99f9db7b23e") # just one page here
connector = NotionConnector(root_page_id=NOTION_DATABASE_ID) # database with pages and tags
connector.load_credentials({"notion_integration_token": NOTION_API_KEY})
connector.is_db

In [ ]:
out = [_ for _ in connector.load_from_state()]

In [ ]:
pages = [connector._fetch_page("a283ff3a46804b39b50e29c566369c2f")]

In [ ]:
parsed_pages = []
for parsed_page in connector._read_pages(pages):
    parsed_pages.append(parsed_page)

In [ ]:
for section in parsed_pages[0]:
    print(section)